In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [3]:

try:
    df_train = pd.read_csv('data/train_processed.csv')
    df_test = pd.read_csv('data/test_processed.csv')
except FileNotFoundError:
    print("FATAL ERROR: Could not find files. Please check file paths.")
    raise

X_train_full = df_train.drop(['has_copd_risk'], axis=1, errors='ignore')
y_train_full = df_train['has_copd_risk']

# Split for Preprocessor fitting (X_train is 80% of data, X_val is 20%)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.2, random_state=42, stratify=y_train_full
)

# 4. Preprocessor Definition and Fit (Median Imputation + Standard Scaling)
all_features = X_train.columns.tolist()

numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, all_features)
    ],
    remainder='passthrough'
)

# Fit preprocessor on training data and transform it
X_train_processed = preprocessor.fit_transform(X_train)

# 5. Train Logistic Regression Model
log_reg = LogisticRegression(solver='liblinear', random_state=42)
log_reg.fit(X_train_processed, y_train)
print("✅ Logistic Regression Model trained successfully.")

# Optional: Evaluate Baseline F1 Score on Validation Set
X_val_processed = preprocessor.transform(X_val)
y_pred_val = log_reg.predict(X_val_processed)
f1 = f1_score(y_val, y_pred_val)
print(f"Validation F1 Score (Baseline): {f1:.4f}")

# ----------------------------------------------------------------------
# C. PREDICT ON TEST DATA AND SAVE SUBMISSION
# ----------------------------------------------------------------------

# 6. Prepare Test Features
test_ids = pd.read_csv('data/ids.csv')
X_test_full = df_test

# Ensure the columns are in the same order as the training features
X_test_full = X_test_full[all_features]

# 7. Apply Preprocessor to Test Data (Transform only!)
X_test_processed = preprocessor.transform(X_test_full)

# 8. Prediction
y_test_pred = log_reg.predict(X_test_processed)

# 9. Combine IDs and Predictions and Save
submission_df = test_ids.copy()
submission_df['has_copd_risk'] = y_test_pred.astype(int)

submission_df.to_csv('submission/logistic_regression.csv', index=False)

print("\nFirst 5 predictions in submission:")
print(submission_df.head())

✅ Logistic Regression Model trained successfully.
Validation F1 Score (Baseline): 0.6672

First 5 predictions in submission:
   patient_id  has_copd_risk
0       42427              0
1       27412              0
2       19283              1
3       45261              1
4       11155              0
